# Dihedral symmetry of the Robinson form

**Adapted from**: Example 5.4 of [GP04]

[GP04] Gatermann, Karin and Parrilo, Pablo A.
*Symmetry groups, semidefinite programs, and sums of squares*.
Journal of Pure and Applied Algebra 192.1-3 (2004): 95-128.

We start by defining the Dihedral group of order 8.
This group is isomorphic to the following permutation group:

In [1]:
using PermutationGroups
d = perm"(1, 2, 3, 4)"
c = perm"(1, 3)"
G = PermGroup([c, d])

Permutation group on 2 generators generated by
 (1,3)
 (1,2,3,4)

We could rely on this isomorphism to define this group.
However, in order to illustrate how to do symmetry reduction with a custom group,
we show in this example what should be implemented to define a new group.

In [2]:
import GroupsCore

struct DihedralGroup <: GroupsCore.Group
    n::Int
end

struct DihedralElement <: GroupsCore.GroupElement
    n::Int
    reflection::Bool
    id::Int
end

Implementing GroupsCore API:

In [3]:
Base.one(G::DihedralGroup) = DihedralElement(G.n, false, 0)

Base.eltype(::Type{DihedralGroup}) = DihedralElement
Base.IteratorSize(::Type{DihedralGroup}) = Base.HasLength()

function Base.iterate(G::DihedralGroup, prev::DihedralElement=DihedralElement(G.n, false, -1))
    if prev.id + 1 >= G.n
        if prev.reflection
            return nothing
        else
            next = DihedralElement(G.n, true, 0)
        end
    else
        next = DihedralElement(G.n, prev.reflection, prev.id + 1)
    end
    return next, next
end

GroupsCore.order(::Type{T}, G::DihedralGroup) where {T} = convert(T, 2G.n)
GroupsCore.gens(G::DihedralGroup) = [DihedralElement(G.n, false, 1), DihedralElement(G.n, true, 0)]

Base.rand not needed for our purposes here

In [4]:
Base.parent(g::DihedralElement) = DihedralGroup(g.n)
function Base.:(==)(g::DihedralElement, h::DihedralElement)
    return g.n == h.n && g.reflection == h.reflection && g.id == h.id
end

function Base.inv(el::DihedralElement)
    if el.reflection || iszero(el.id)
        return el
    else
        return DihedralElement(el.n, false, el.n - el.id)
    end
end
function Base.:*(a::DihedralElement, b::DihedralElement)
    a.n == b.n || error("Cannot multiply elements from different Dihedral groups")
    id = mod(a.reflection ? a.id - b.id : a.id + b.id, a.n)
    return DihedralElement(a.n, a.reflection != b.reflection, id)
end

Base.copy(a::DihedralElement) = DihedralElement(a.n, a.reflection, a.id)

optional functions:

In [5]:
function GroupsCore.order(el::DihedralElement)
    if el.reflection
        return 2
    else
        if iszero(el.id)
            return 1
        else
            return div(el.n, gcd(el.n, el.id))
        end
    end
end

The Robinson form is invariant under the following action of the Dihedral group on monomials:
The action of each element of the groups is to map the variables `x, y` to:

| id | rotation | reflection |
|----|----------|------------|
| 0  | x, y     | y, x       |
| 1  | -y, x    | -x, y      |
| 2  | -x, -y   | -y, -x     |
| 3  | y, -x    | x, -y      |

In [6]:
using SumOfSquares
using DynamicPolynomials
@polyvar x y
struct DihedralAction <: Symmetry.OnMonomials end
import SymbolicWedderburn
SymbolicWedderburn.coeff_type(::DihedralAction) = Float64
function SymbolicWedderburn.action(::DihedralAction, el::DihedralElement, mono::AbstractMonomial)
    if iseven(el.reflection + el.id)
        var_x, var_y = x, y
    else
        var_x, var_y = y, x
    end
    sign_x = 1 <= el.id <= 2 ? -1 : 1
    sign_y = 2 <= el.id ? -1 : 1
    return mono([x, y] => [sign_x * var_x, sign_y * var_y])
end

poly = x^6 + y^6 - x^4 * y^2 - y^4 * x^2 - x^4 - y^4 - x^2 - y^2 + 3x^2 * y^2 + 1

x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1

We can verify that `poly` is indeed invariant under the action of each element of the group as follows.

In [7]:
G = DihedralGroup(4)
for g in G
    @show SymbolicWedderburn.action(DihedralAction(), g, poly)
end

SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1


We can exploit this symmetry for reducing the problem using the `SymmetricIdeal` certificate as follows:

In [8]:
import CSDP
function solve(G)
    solver = CSDP.Optimizer
    model = Model(solver)
    @variable(model, t)
    @objective(model, Max, t)
    pattern = Symmetry.Pattern(G, DihedralAction())
    con_ref = @constraint(model, poly - t in SOSCone(), symmetry = pattern)
    optimize!(model)
    @show value(t)


    for g in gram_matrix(con_ref).sub_gram_matrices
        println(g.basis.polynomials)
    end
end
solve(G)

Iter: 17 Ap: 9.60e-01 Pobj: -3.5512010e-03 Ad: 9.60e-01 Dobj: -3.5513625e-03 
Success: SDP solved
Primal objective value: -3.5512010e-03 
Dual objective value: -3.5513625e-03 
Relative primal infeasibility: 8.88e-14 
Relative dual infeasibility: 1.65e-09 
Real Relative Gap: -1.60e-07 
XZ Relative Gap: 2.91e-09 
DIMACS error measures: 9.58e-14 0.00e+00 4.44e-09 0.00e+00 -1.60e-07 2.91e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 7.29e-01 Pobj: -3.4673594e+00 Ad: 8.60e-01 Dobj: -3.3762458e-01 
Iter:  2 Ap: 7.63e-01 Pobj: -1.3183897e+01 Ad: 9.55e-01 Dobj:  7.9764348e+00 
Iter:  3 Ap: 8.98e-01 Pobj: -5.7688875e+00 Ad: 7.59e-01 Dobj:  4.6876367e+00 
Iter:  4 Ap: 8.22e-01 Pobj: -3.0277321e+00 Ad: 6.68e-01 Dobj:  1.3865530e+00 
Iter:  5 Ap: 7.08e-01 Pobj: -1.7157542e+00 Ad: 7.92e-01 Dobj: -8.7074774e-02 
Iter:  6 Ap: 7.96e-01 Pobj: -1.2594928e+00 Ad: 7.47e-01 Dobj: -5.9489850e-01 
Iter:  7 Ap: 7.77e-01 Pobj: -1.0320698e+00 Ad: 7.76e

We notice that we indeed find `-3825/4096` and that symmetry was exploited.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*